In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import zarr
import plotter as plo
from cmcrameri import cm
import pywtt
import scipy.stats
from matplotlib.gridspec import GridSpec
import matplotlib
import CHONK_XL as chxl
from cmcrameri import cm
from matplotlib.gridspec import GridSpec


matplotlib.rc('font', family='Helvetica Neue') 
matplotlib.rc('font', serif='Helvetica Neue') 
matplotlib.rcParams.update({'font.size': 16})

%matplotlib widget
def divide_labels(ax, divider, isint = True, axis = "both"):
    if(axis == "both"):
        onx = True
        ony = True
    elif (axis == "x"):
        onx = True
        ony = False
    elif (axis == "y"):
        onx = False
        ony = True
            
    
    xticks = ax.get_xticks()
    yticks = ax.get_yticks()

    xtickslab = ax.get_xticklabels()
    ytickslab = ax.get_yticklabels()
    if(onx):
        for i in range(len(xticks)):
            xtickslab[i] = str(int(xticks[i]/divider)) if isint else str(xticks[i]/divider)
        ax.set_xticklabels(xtickslab)
    
    if(ony):
        for i in range(len(yticks)):
            ytickslab[i] = str(int(yticks[i]/divider)) if isint else str(yticks[i]/divider)
        ax.set_yticklabels(ytickslab)

In [ ]:
ds = xr.open_zarr('section_tracking_simple2.zarr')
ds.x.values[0] = 0
ds.y.values[0] = 0
ny = ds.ChonkBase__ny.item(0)
nx = ds.ChonkBase__nx.item(0)

extent = [ds.x.min(),ds.x.max(),ds.y.max(), ds.y.min() ]
extent1 = [ds.x.min(),ds.x.max()/2,ds.y.max(), ds.y.min() ]
extent2 = [ds.x.max()/2,ds.x.max(),ds.y.max(), ds.y.min() ]
sel = {"otime": 1e7}
label = 1
ttopo = ds.Topography__topography.sel({'otime':sel['otime']}, method = 'nearest').values
HS = pywtt.minigraph(ds.ChonkBase__nx.values.item(0), ds.ChonkBase__ny.values.item(0), ds.ChonkBase__nx.values.item(0) * ds.ChonkBase__ny.values.item(0), ds.ChonkBase__dx, ds.ChonkBase__dy, 0, 0, ttopo.ravel()).get_HS()
HS[np.isfinite(HS) == False] = 0
HS[HS < 80] = 80
HS = HS - 80
HS = HS / HS.max()
np.unique(HS)



In [ ]:
sumVol = chxl.get_full_volume_sediment_label_map(ds,sel,label, ChonkBase = 'ChonkBase')
sumVol10m = chxl.get_first_m_volume_sediment_label_map(ds,sel,label,10, ChonkBase = 'ChonkBase')

In [ ]:
plt.close('all')
fig = plt.figure(figsize = (8,5))


gs = GridSpec(100, 100, figure=fig)

ax = fig.add_subplot(gs[:,25:90])
axcb1 = fig.add_subplot(gs[8:92,:7])
axcb2 = fig.add_subplot(gs[10:90,93:99])

cb1 = ax.imshow(sumVol[:,:100], extent = extent1, cmap = cm.tokyo_r, zorder = 1)
cb2 = ax.imshow(sumVol10m[:,100:], extent = extent2, cmap = cm.tokyo_r, zorder = 1) #tokyo, lapaz

ax.set_xlim(extent[0],extent[1])
ax.set_ylim(extent[2],extent[3])
plt.colorbar(cb1, cax=axcb1, label = "Full volume in basins")
ax.imshow(HS.reshape(ny,nx), extent = extent, cmap = cm.grayC_r, alpha = 0.35, zorder = 2)
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
plt.colorbar(cb2, cax=axcb2, label = "Volume in the first 10 m")
ax.axvline(ds.x.max()/2, lw = 2, color = 'k')

In [ ]:
ds.n_labels

In [ ]:
csel = sel.copy()
csel["y"] = 30000
fig, ax = chxl.get_cross_section(
	ds, # The input ds 
	# fname = "well.png",
	csel,
	ChonkBase = 'ChonkBase',
	label = 1,
	cmap = cm.tokyo_r,
	color_bedrock = 'gray',
	figsize = None,
	nlabels = 2,
	dpi = 300,
    direction = "EW"
    # minmax = (32000,42000)

)
ax.grid(ls = '--', lw = 0.5, color = 'k', alpha = 0.6)

In [ ]:
fig, ax = chxl.get_well_time_serie(
	ds, # The input ds 
	# fname = "well.png",
	timedim = "otime", # the time dimension
	X = 27000,
	Y = 31000,
	ChonkBase = 'ChonkBase',
	label = 1,
	cmap = cm.tokyo_r,
	color_bedrock = 'gray',
	figsize = None,
	nlabels = 2,
	dpi = 300,
    minmax = [0,0.25]
    
)

xticks = ax.get_xticks()[1:-1] #< notice the slicing
yticks = ax.get_yticks()[1:-1] #< notice the slicing
xgrid, ygrid = np.meshgrid(xticks, yticks)
ax.scatter(xgrid,ygrid, marker= "+",c = 'k', s = 20, lw = 1 , alpha = 0.4, zorder = 5)
divide_labels(ax,1e6, axis='x')
ax.set_xlabel("Time (Myrs)")
plt.savefig("/Users/gailleto/Desktop/papers/CHONK_!/figure_tracking/fig2.svg")
plt.savefig("/Users/gailleto/Desktop/papers/CHONK_!/figure_tracking/fig2.pdf")
plt.savefig("/Users/gailleto/Desktop/papers/CHONK_!/figure_tracking/fig2.png")

In [ ]:
# fig, ax = chxl.get_cross_section_EW_props(
# 	ds, # The input ds 
# 	# fname = "well.png",
# 	csel,
# 	ChonkBase = 'ChonkBase',
# 	label = 1,
# 	cmap = cm.bam,
# 	color_bedrock = 'gray',
# 	figsize = None,
# 	nlabels = 2,
# 	dpi = 300,
#     # minmax = (32000,42000)

# )

In [ ]:
# arr.shape

In [ ]:
# fig,ax = plt.subplots()
# ax.hist(HS)

In [ ]:
from ipyfastscape import TopoViz3d


app = TopoViz3d(ds, canvas_height=600, time_dim="otime", elevation_var = "Topography__topography" )

app.show()